Este proyecto se compone de distintos módulos que contienen las diferentes líneas de investigación que se han llevado a cabo para el desarrollo de este cuaderno. 

En las respectivas carpetas se encontrarás archivos `.py` y `jupyter notebooks` junto con sus respectivas carpetas anidadas `data` que contienen los datos utilizados en cada módulo.

# **PIANO SOUL**

## Idea inicial

#### PianoSoul es un proyecto que nace de la idea de querer representar musicalmente (en nuestro caso con un piano) cómo nos sentimos. 
#### El objetivo principal era, que el usuario que utilice PianoSoul, escribiera un texto y este fuera analizado por un LLM. Con esto se obtendrían unas características que posteriormete se le proporcionarían a un modelo y este generaría una melodía, cadencia o una pequeña composición musical para dicho usuario.
<br>

---

## Desarrollo

### 1. Primer módulo &rightarrow; `project_develp`


Este fue el primer módulo del proyecto que implementamos. 

Partimos de un conjunto de datos sobre fragmentos musicales encontrados en [Kaggle](https://cvml.unige.ch/databases/DEAM/). De estos datos utilizamos el .cvs con datos sobre la positividad y la activación sobre los fragmentos musicales analizados y la carpeta con los respectivos audios (mp3).


Empezamos el proyecto buscando en internet cómo poder etiquetar los datos que teníamos con unos sentimientos de manera objetiva, sin interferir nosotros sesgando las emociones de manera individual.
Encontramos que podíamos definir las emociones **científicamente** basándonos en el modelo de [**circunflejo emocional**](https://psicologiaingresouna.wordpress.com/contenidos/conducta-motivada-y-emocional/emociones/) (Russell, 1980), que mapea emociones en un plano bidimensional con los ejes de **palcer/displacer** (positividad) y **activación/inactividad**.



<p align="center">
  <img src="https://psicologiaingresouna.wordpress.com/wp-content/uploads/2018/01/circumplex-of-russell1.png" alt="Circumplex of Russell">
</p>



#### **Rangos de Valores**
- **Valence** (positividad):
  - \(-1\): Muy negativo.
  - \(0\): Neutral.
  - \(+1\): Muy positivo.
- **Arousal** (activación):
  - \(-1\): Muy calmado.
  - \(0\): Neutral.
  - \(+1\): Muy activo.

#### **Distribución de Emociones en el Espacio**
Basándonos en combinaciones de valence y arousal, podemos asignar emociones más detalladas:

| **Valence**              | **Arousal**                    | **Emotion**       |
|---------------------------|---------------------------------|-------------------|
| Muy positivo > 0.5        | Muy activo > 0.5               | **Excitement**    |
| Muy positivo > 0.5        | Neutral entre 0.0 y 0.5        | **Joy**           |
| Muy positivo > 0.5        | Muy calmado < 0.0              | **Contentment**   |
| Neutral entre 0.0 y 0.5   | Muy activo > 0.5               | **Elation**       |
| Neutral entre 0.0 y 0.5   | Neutral entre 0.0 y 0.5        | **Happiness**     |
| Neutral entre 0.0 y 0.5   | Muy calmado < 0.0              | **Calmness**      |
| Muy negativo entre -0.5 y 0.0 | Muy activo > 0.5            | **Stress**        |
| Muy negativo entre -0.5 y 0.0 | Neutral entre 0.0 y 0.5    | **Satisfaction**  |
| Muy negativo entre -0.5 y 0.0 | Muy calmado < 0.0          | **Relaxation**    |
| Muy negativo < -0.5       | Muy activo > 0.5               | **Fear**          |
| Muy negativo < -0.5       | Neutral entre 0.0 y 0.5        | **Anxiety**       |
| Muy negativo < -0.5       | Muy calmado < 0.0              | **Sadness**       |
| Muy negativo < -0.5       | Muy calmado extremo < -0.5     | **Depression**    |
| Muy negativo < -0.5       | Activación intermedia entre -0.5 y 0.0 | **Anger**      |
| Neutral cualquier valence | Muy inactivo < -0.7            | **Sleep**         |
| Cualquier valor           | Sin clasificación específica   | **Neutral**       |

---

Tras hacer una limpieza de los valores del CSV, creamos una función `map_emotion` a la que le pasamos los datos del dataset y concatenamos las emociones en una nueva columna (puedes encontrar el dataset final usado en el archivo `features_val_ar_emotions.csv` dentro de la carpeta data).

Obtenemos este gráfico de forma visual apra orientarnos:

<br>
<p align="center">
    <img src="project_develp\data\output_sentimientos.png" alt="grafico emociones mapeo" width="800">
</p>
<br>

--- 

Después, buscamos qué librería nos podía servir para analizar audio. Elegimos **librosa** para procesar archivos .mp3, para todo el tema de las características muscicales. Sin embargo, esta librería tenía una imitación, no permitía obtener datos conceptuales como notas o tonalidades y sólo permitía la extracción de datos estructurales como tempo o ritmo. 

A pesar de que esto nos suponía una limitación muy grande, seguimos con el desarrollo teniendo la esperanza de que el modelo predijera bien esos "datos estructurales" para luego pasárselo a un modelo que generase música. 


Para entrenar el modelo encodeamos (utilizando `pd.get_dummies(df['emotion'], prefix='emotion')`) los sentimientos utilizados en la función de mapeo para que tuvieran todo en forma matricial con 1 y 0.
 
Sin embargo, una vez entrenado el modelo, obtenimos un `mse: 53911182.310126126` y un `r2 score: 0.17416793955996174`, lo que supone un resultado **nefasto**. 

Como además las características que nos daba librosa no nos servían mucho para generar música, **decidimos abandonar esta línea del proyecto y explorar otras ramas.**

---

<br>

### 2. Segundo módulo &rightarrow; `obtencion_acordes`

### 3. Tercer módulo &rightarrow; `piano-webpage`

### 4. Cuarto módulo &rightarrow; `api_database`


Esta es la última línea de desarrollo que hemos seguido.

Tras ver el potencial que tiene, para nuestro objetivo, la página web, decidimos emplear un **Modelo de Lenguaje de Largo alcance** para procesar el texto del usuario de `PianoSoul`. 

Implementamos una API personalizada  de [**GoogleAi**](https://ai.google.dev/gemini-api/docs/quickstart?hl=es-419&lang=python) con unos conocimientos adquiridos previamente diseñado por nosotros.

---

Construimos una base de datos en formato `json` con la información de distintas páginas webs que hemos buscado y leído. 

Con la ayuda de librerías como `BeautifulSoup`, obtenemos el contenido de dentro de las etiquetas HTML constuirlo de manera más estructurada. 

Guardamos este texto en formato json. A cada "valor del diccionario" le aplicamos un proceso de preprocesado con `regex` para eliminar urls, etiquetas html, caracteres no alfanuméricos y espacios innecesarios. Mantenemos los siggnos de puntuación para procesar un contexto completo.

Como no encontramos nada muy explícito acerca de los modos musicales, le pedimos a chatgpt que nos hardcodeara una pequeña base de conocimientos sobre modos musicales y sentimientos. Esto también se lo pasaremso a la API

---

Para pasar al proceso de construcción de la API usamos la guía básica de googleai proporiconada en el enlace. 

Nos sacamso nuestra Api key (única e intrasferible) y pasamos a diseñar el prompt que posteriormente procesará.

Para meter en contexto a la IA, le indicamos que actúe como un experto en música y psicología especializada en la relación entre música y emociones. Su tarea es analizar un texto proporcionado por el usuario, comprender el estado emocional subyacente y traducirlo en características musicales específicas, proporcionando un dictado detallado con valores útiles para componer música.

Estas **características musicales** las hemos decidido bajo nuestro criterio de 'personas que tocan un isntrumento' relacionándolo también con lo leído en las páginas web. Son las siguientes: **Tempo, Intensidad/Dinámica, Timbre, Ritmo, Progresión armónica, Melodía, Tonalidad/Modo, Articulación, Silencio.**

---

En el respectivo notebook `api_devel.ipynb` podrás encontrar las respuestas obtenidas a nuestro "user texts" e incluso probarlo tú mismo.

Nosotros hemso probado como sonaba en el piano y nos ha parecido suficientemente acertado para tratarse de algo tan básico.

Esto nos abre nuevas líneas de investigación que quedan como futuras ramas abiertas a continuar.

## Futuras líneas de investigación:

### - Diseñar nuestro propio LLM 
&nbsp; y entrenarlo para darle un enfoque más ajustado.
### - Desarrollar un modelo basado en redes de neuronas 
&nbsp; que mejores el RandomForest que hemos usado.
### - Mapear el sentimiento de los lyrics del dataset con las canciones y cadencias 
&nbsp; para así obtener un conjunto de datos etiquetados basado en análisis de sentimientos en textos.
### - Enlazar la respuesta de la API con el piano interactivo
&nbsp; para así poder reproducir la cadencia en la tonalidad deseada en el piano y poder inprovisar una pequeña melodía o probar la generada por la api